In [ ]:
import pandas as pd

# Load the Excel file
file_path = "C:\\Users\\User\\Downloads\\DA -Task 2..xlsx"
df = pd.read_excel(file_path)

# Collect column-wise information
summary = []
for col in df.columns:
    data_type = df[col].dtype
    missing_pct = df[col].isnull().mean() * 100
    unique_vals = df[col].nunique()
    example_vals = df[col].dropna().unique()[:3]
    
    summary.append({
        "Column Name": col,
        "Data Type": str(data_type),
        "Missing %": round(missing_pct, 2),
        "Unique Values": unique_vals,
        "Example Values": example_vals
    })

# Create a DataFrame summary
df_summary = pd.DataFrame(summary)

# Save to Excel
df_summary.to_excel("C:\\Users\\User\\Downloads\\task2.xlsx", index=False)


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore


# Load the data
file_path ="C:\\Users\\User\\Downloads\\DA -Task 2..xlsx"
df = pd.read_excel(file_path)

# Step 1: View basic info and nulls
print("Initial Shape:", df.shape)
print("\nMissing Values Per Column:\n", df.isnull().sum())

# Step 2: Drop columns with more than 50% missing values
threshold = 0.5 * len(df)
df = df.loc[:, df.isnull().sum() < threshold]
print("\nShape after dropping high-null columns:", df.shape)

# Step 3: Fill remaining missing values
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna("Unknown")
    else:
        df[col] = df[col].fillna(df[col].median())

# Step 4: Clean categorical columns (standardize strings)
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col] = df[col].str.lower().str.strip()

# Step 5: Convert numeric columns to proper format
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            continue

# Step 6: Remove outliers using Z-score (optional, skip if not needed)
numeric_cols = df.select_dtypes(include=np.number).columns
df = df[(np.abs(zscore(df[numeric_cols])) < 3).all(axis=1)]
print("\nShape after outlier removal:", df.shape)

# Step 7: Save cleaned data
output_path = "C:\\Users\\User\\Downloads\\task2.xlsx"

output_sheet_name = "cleansing_Output"

# Use openpyxl engine to modify existing Excel file and write to a new sheet
with pd.ExcelWriter(output_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=output_sheet_name, index=False)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage
from PIL import Image

# Step 1: Load the dataset
file_path  = "C:\\Users\\User\\Downloads\\DA -Task 2..xlsx"
df = pd.read_excel(file_path)

# Step 2: Clean column names
df.columns = df.columns.str.lower().str.replace(" ", "_").str.strip()

# Check actual column names
print("Available columns:\n", df.columns.tolist())

# Step 3: Define columns for plots (make sure they exist)
# Replace column names if different in your data
plot_cols = {
    'causal_part_nm': 'Top 10 Causal Parts',
    'vin_modl_desgtr': 'Top 10 VIN Models',
    'totalcost': 'Total Cost Distribution'
}

# Step 4: Create plots and save as images
plots = []
for col, title in plot_cols.items():
    plt.figure(figsize=(10, 5))
    
    if col == 'totalcost':
        sns.histplot(df[col], bins=30, kde=True)
        plt.xlabel('Total Cost')
        plt.ylabel('Frequency')
    else:
        sns.countplot(data=df, y=col, order=df[col].value_counts().index[:10])
        plt.xlabel("Count")
        plt.ylabel(col)

    plt.title(title)
    filename = f"{col}_plot.png"
    plt.tight_layout()
    plt.savefig(filename)
    plots.append(filename)
    plt.close()

# Step 5: Save plots to a new Excel workbook
wb = Workbook()
ws = wb.active
ws.title = "Visualizations"

# Add images to Excel sheet
for i, plot in enumerate(plots, start=1):
    img = XLImage(plot)
    img.anchor = f"A{(i - 1) * 20 + 1}"  # Spacing between images
    ws.add_image(img)

# Save workbook
output_excel = "C:\\Users\\User\\Downloads\\task2.1.xlsx"
wb.save(output_excel)
print(f"Visual report saved to {output_excel}")

# Optional: clean up .png files
for plot in plots:
    os.remove(plot)
